<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/Distilbert_FineTune_Intent_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://www.kaggle.com/datasets/himanshunayal/intent-recognition-dataset?select=train.csv

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import torch
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
df=pd.read_csv("/content/train.csv")

In [5]:
df

,text,intent
0,listen to westbam alumb allergic on google music,PlayMusic
1,add step to me to the 50 clásicos playlist,AddToPlaylist
2,i give this current textbook a rating value of...,RateBook
3,play the song little robin redbreast,PlayMusic
4,please add iris dement to my playlist this is ...,AddToPlaylist
...,...,...
13079,i want to eat choucroute at a brasserie for 8,BookRestaurant
13080,play funky heavy bluesy,PlayMusic
13081,rate the current album 2 points out of 6,RateBook
13082,go to the photograph the inflated tear,SearchCreativeWork


In [6]:
df['intent'].value_counts()

,count
intent,
PlayMusic,1914
GetWeather,1896
BookRestaurant,1881
RateBook,1876
SearchScreeningEvent,1852
SearchCreativeWork,1847
AddToPlaylist,1818


In [7]:
df.isnull().sum()

,0
text,0
intent,0


In [8]:
df['intent']=df['intent'].str.lower()

In [9]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words=set(stopwords.words('english'))

def preprocess_text(text):
  text=text.lower()
  text=text.split()
  # remove stopwords
  text = ' '.join([word for word in text if word not in stop_words])
  return text


text="This is LLM model implementation"
preprocess_text(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'llm model implementation'

In [10]:
df['text']=df['text'].apply(preprocess_text)

In [11]:
df

,text,intent
0,listen westbam alumb allergic google music,playmusic
1,add step 50 clásicos playlist,addtoplaylist
2,give current textbook rating value 1 best rati...,ratebook
3,play song little robin redbreast,playmusic
4,please add iris dement playlist selena,addtoplaylist
...,...,...
13079,want eat choucroute brasserie 8,bookrestaurant
13080,play funky heavy bluesy,playmusic
13081,rate current album 2 points 6,ratebook
13082,go photograph inflated tear,searchcreativework


In [12]:
unique_labels=df['intent'].unique()
print("unique labels:",pd.unique)

label_to_id={label:i for i , label in enumerate(unique_labels)}

df['label']=df['intent'].map(label_to_id)
df.head()

unique labels: <function unique at 0x78ca89f46e60>


,text,intent,label
0,listen westbam alumb allergic google music,playmusic,0
1,add step 50 clásicos playlist,addtoplaylist,1
2,give current textbook rating value 1 best rati...,ratebook,2
3,play song little robin redbreast,playmusic,0
4,please add iris dement playlist selena,addtoplaylist,1


In [13]:
tokenizer=DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [14]:
max_length=max([len(tokenizer.encode(text)) for text in df['text']])
print(max_length)

28


In [15]:
# Tokenization
def tokenize_function(example):
  tokenized_input=tokenizer(example['text'],padding='max_length',truncation=True,max_length=max_length)
  tokenized_input['labels']=example['label']
  return tokenized_input

dataset=Dataset.from_pandas(df[['text','label']])
dataset=dataset.map(tokenize_function,batched=True)

dataset[0]

Map:   0%|          | 0/13084 [00:00<?, ? examples/s]

{'text': 'listen westbam alumb allergic google music',
 'label': 0,
 'input_ids': [101,
  4952,
  2225,
  3676,
  2213,
  2632,
  25438,
  27395,
  8224,
  2189,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'labels': 0}

In [16]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=len(unique_labels))

import torch
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [21]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [22]:
training_arguments=TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer=Trainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset,
    eval_dataset=dataset,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.037500,0.031689
2,0.002300,0.019579
3,0.048200,0.010090
4,0.001000,0.004652
5,0.000400,0.003022


TrainOutput(global_step=4090, training_loss=0.0635745094789643, metrics={'train_runtime': 350.9121, 'train_samples_per_second': 186.428, 'train_steps_per_second': 11.655, 'total_flos': 473965075426080.0, 'train_loss': 0.0635745094789643, 'epoch': 5.0})

In [23]:
# Evaluate
predictions,true_labels,_=trainer.predict(dataset)
predicted_labels=predictions.argmax(axis=1)

print("Classificatio Report")
print(classification_report(true_labels,predicted_labels))


print("Confusion matrix:")
print(confusion_matrix(true_labels,predicted_labels))

Classificatio Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1914
           1       1.00      1.00      1.00      1818
           2       1.00      1.00      1.00      1876
           3       1.00      1.00      1.00      1852
           4       1.00      1.00      1.00      1881
           5       1.00      1.00      1.00      1896
           6       1.00      1.00      1.00      1847

    accuracy                           1.00     13084
   macro avg       1.00      1.00      1.00     13084
weighted avg       1.00      1.00      1.00     13084

Confusion matrix:
[[1914    0    0    0    0    0    0]
 [   1 1817    0    0    0    0    0]
 [   0    0 1876    0    0    0    0]
 [   0    0    0 1851    0    0    1]
 [   0    0    0    0 1881    0    0]
 [   0    0    0    0    3 1893    0]
 [   0    0    0    3    0    0 1844]]


In [24]:
model.save_pretrained("./intent_classification_model")
tokenizer.save_pretrained("./intent_classification_model")

('./intent_classification_model/tokenizer_config.json',
 './intent_classification_model/special_tokens_map.json',
 './intent_classification_model/vocab.txt',
 './intent_classification_model/added_tokens.json')

In [25]:
model = DistilBertForSequenceClassification.from_pretrained('./intent_classification_model')
tokenizer = DistilBertTokenizer.from_pretrained('./intent_classification_model')

In [30]:
import torch

text = "Play Some Awesome music"

id_to_label = {v: k for k, v in label_to_id.items()}  # Reverse mapping for label prediction

def preprocess_text(text):
    # Implement your preprocessing function here if it’s not defined
    return text.lower()  # Example preprocessing to lowercase

# Use the predict function to get the predicted label
predicted_label = predict(text, model, tokenizer, max_length=max_length)
print(f"Predicted Label: {predicted_label}")

Predicted Label: playmusic
